In [1]:
import pandas as pd
from rake_nltk import Rake

import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer



In [2]:
#loading the dataset
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df = df[['Title','Genre','Director','Actors','Plot']]

df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [4]:
#convert to csv file
df.to_csv('df_old.csv', index=False) 

In [19]:
df= pd.read_csv(r'C:\Users\onfon\Desktop\recommender system\df_old.csv')
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


# Extracting the key words from the plot description.

In [20]:
# initializing the new column

df['Key_words'] = ""



for index, row in df.iterrows():

    plot = row['Plot']

    

    # instantiating Rake, by default it uses english stopwords from NLTK

    # and discards all puntuation characters as well

    r = Rake()



    # extracting the words by passing the text

    r.extract_keywords_from_text(plot)



    # getting the dictionary whith key words as keys and their scores as values

    key_words_dict_scores = r.get_word_degrees()

    

    # assigning the key words to the new column for the corresponding movie

    row['Key_words'] = list(key_words_dict_scores.keys())





In [21]:
# dropping the Plot column

df.drop(columns = ['Plot'], inplace = True)

In [22]:
#dropna

df.dropna() 

,Title,Genre,Director,Actors,Key_words
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","[eventual, redemption, number, acts, finding, ..."
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[reluctant, son, aging, patriarch, organized, ..."
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...","[grip, expands, career, family, crime, syndica..."
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","[mysterious, past, greatest, psychological, jo..."
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....","[forcing, reconsider, evidence, justice, preve..."
5,Schindler's List,"Biography, Drama, History",Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...","[occupied, poland, oskar, schindler, gradually..."
6,The Lord of the Rings: The Return of the King,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston","[draw, gaze, frodo, sauron, men, aragorn, lead..."
7,Pulp Fiction,"Crime, Drama",Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...","[violence, wife, four, tales, pair, gangster, ..."
8,Fight Club,Drama,David Fincher,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier","[way, devil, underground, fight, club, life, c..."
9,The Lord of the Rings: The Fellowship of the Ring,"Adventure, Drama, Fantasy",Peter Jackson,"Alan Howard, Noel Appleby, Sean Astin, Sala Baker","[journey, dark, lord, sauron, eight, companion..."


In [23]:
df['Actors']=df['Actors'].astype('str')
df['Genre']=df['Genre'].astype('str')
df['Director']=df['Director'].astype('str')

# Transforming the full names of actors and directors in single words so they are considered as unique values.

In [24]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

# putting the genres in a list of words
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

df['Director'] = df['Director'].map(lambda x: x.split(' '))

In [25]:
# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [29]:
for index, row in df.iterrows():
    row['Genre'] = ''.join(row['Genre']).lower()

KeyError: 'Genre'

In [26]:
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[eventual, redemption, number, acts, finding, ..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[reluctant, son, aging, patriarch, organized, ..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[grip, expands, career, family, crime, syndica..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[mysterious, past, greatest, psychological, jo..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[forcing, reconsider, evidence, justice, preve..."


# creating bag of words

In [27]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [28]:
df.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...


In [11]:
#convert to csv file
df.to_csv('df.csv', index=False) 

In [30]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

In [31]:
count_matrix.shape

(250, 2961)

In [32]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list to be used later to match the indexes
indices = pd.Series(df.index)
indices

0                               The Shawshank Redemption
1                                          The Godfather
2                                 The Godfather: Part II
3                                        The Dark Knight
4                                           12 Angry Men
5                                       Schindler's List
6          The Lord of the Rings: The Return of the King
7                                           Pulp Fiction
8                                             Fight Club
9      The Lord of the Rings: The Fellowship of the Ring
10                                          Forrest Gump
11        Star Wars: Episode V - The Empire Strikes Back
12                                             Inception
13                 The Lord of the Rings: The Two Towers
14                       One Flew Over the Cuckoo's Nest
15                                            Goodfellas
16                                            The Matrix
17                    Star Wars

In [33]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.15789474, 0.13764944, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.15789474, 1.        , 0.36706517, ..., 0.05263158, 0.05263158,
        0.05564149],
       [0.13764944, 0.36706517, 1.        , ..., 0.04588315, 0.04588315,
        0.04850713],
       ...,
       [0.05263158, 0.05263158, 0.04588315, ..., 1.        , 0.05263158,
        0.05564149],
       [0.05263158, 0.05263158, 0.04588315, ..., 0.05263158, 1.        ,
        0.05564149],
       [0.05564149, 0.05564149, 0.04850713, ..., 0.05564149, 0.05564149,
        1.        ]])

In [34]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [36]:
df.dtypes

bag_of_words    object
dtype: object

In [35]:

recommendations('Fargo')

['No Country for Old Men',
 'The Departed',
 'Rope',
 'The Godfather',
 'Reservoir Dogs',
 'The Godfather: Part II',
 'On the Waterfront',
 'Goodfellas',
 'Arsenic and Old Lace',
 'The Big Lebowski']

In [36]:

recommendations('The Shawshank Redemption')

['Pulp Fiction',
 'Rope',
 'Se7en',
 'Goodfellas',
 'Butch Cassidy and the Sundance Kid',
 'The Godfather',
 'Papillon',
 'The Green Mile',
 'Casino',
 'Once Upon a Time in America']

In [24]:
recommendations('Harvey')

['The Kid',
 'It Happened One Night',
 'American Beauty',
 'Modern Times',
 'Arsenic and Old Lace',
 "It's a Wonderful Life",
 'Mr. Smith Goes to Washington',
 'The Apartment',
 'Gone Girl',
 'The Green Mile']

In [25]:
recommendations('Slumdog Millionaire')

['Trainspotting',
 'The Green Mile',
 'Goodfellas',
 'Lion',
 'Room',
 'Cool Hand Luke',
 "Hachi: A Dog's Tale",
 'Brief Encounter',
 'La La Land',
 'The Graduate']

In [62]:
df.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...


# Articles

In [84]:
df2 = pd.read_csv(r'C:\Users\onfon\Desktop\recommender system\articles1.csv')
df2.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [85]:
df2 = df2[['title','publication','author','content']]
df2.head()

,title,publication,author,content
0,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,"Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,"SEOUL, South Korea — North Korea’s leader, ..."


In [87]:
df2.dropna()   
df2.author.value_counts()

Breitbart News                                                   1559
Pam Key                                                          1282
Charlie Spiering                                                  928
Jerome Hudson                                                     806
John Hayward                                                      747
Daniel Nussbaum                                                   735
AWR Hawkins                                                       720
Ian Hanchett                                                      647
Joel B. Pollak                                                    624
Alex Swoyer                                                       604
Warner Todd Huston                                                545
Jeff Poor                                                         505
Trent Baker                                                       457
Breitbart London                                                  447
Katherine Rodriguez 

In [93]:
 df2['author'].astype(str)

0                            Carl Hulse
1         Benjamin Mueller and Al Baker
2                          Margalit Fox
3                      William McDonald
4                         Choe Sang-Hun
5                           Sewell Chan
6                   Javier C. Hernández
7                           Gina Kolata
8                      Katherine Rosman
9                           Andy Newman
10                        Justin Gillis
11                        John Schwartz
12                      Maggie Haberman
13                       Charles Duhigg
14                 Stephanie Rosenbloom
15                  Emma G. Fitzsimmons
16          Kevin Sack and Alan Blinder
17                          Geeta Anand
18                 The Associated Press
19                           Brett Cole
20                     Benjamin Hoffman
21                        Patrick Healy
22                       Marlise Simons
23       Guy Trebay and Jacob Bernstein
24                  Jennifer Steinhauer


In [90]:
df2.dtypes

title          object
publication    object
author         object
content        object
dtype: object

In [66]:
type(df2['author'])

pandas.core.series.Series

In [80]:
df2.dtypes

Unnamed: 0       int64
id               int64
title           object
publication     object
author          object
date            object
year           float64
month          float64
url            float64
content         object
dtype: object

In [79]:
df2['author']=df2['author'].astype('str')

In [94]:
df2['author'] = df2['author'].map(lambda x: x.split(' '))

In [99]:
df2['author'].head(15)

0                           [Carl, Hulse]
1     [Benjamin, Mueller, and, Al, Baker]
2                         [Margalit, Fox]
3                     [William, McDonald]
4                        [Choe, Sang-Hun]
5                          [Sewell, Chan]
6                 [Javier, C., Hernández]
7                          [Gina, Kolata]
8                     [Katherine, Rosman]
9                          [Andy, Newman]
10                       [Justin, Gillis]
11                       [John, Schwartz]
12                     [Maggie, Haberman]
13                      [Charles, Duhigg]
14                [Stephanie, Rosenbloom]
Name: author, dtype: object

In [96]:
df2['publication'] = df2['publication'].map(lambda x: x.split(' '))

In [97]:
df2['publication'].head(12)

0     [New, York, Times]
1     [New, York, Times]
2     [New, York, Times]
3     [New, York, Times]
4     [New, York, Times]
5     [New, York, Times]
6     [New, York, Times]
7     [New, York, Times]
8     [New, York, Times]
9     [New, York, Times]
10    [New, York, Times]
11    [New, York, Times]
Name: publication, dtype: object

In [100]:
# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in df2.iterrows():
    row['publication'] = [x.lower().replace(' ','') for x in row['publication']]
    row['author'] = ''.join(row['author']).lower()

In [102]:
for index, row in df2.iterrows():
    row['publication'] = ''.join(row['publication']).lower()

In [103]:
df2.head()

,title,publication,author,content
0,House Republicans Fret About Winning Their Hea...,newyorktimes,carlhulse,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,newyorktimes,benjaminmuellerandalbaker,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",newyorktimes,margalitfox,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",newyorktimes,williammcdonald,"Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,newyorktimes,choesang-hun,"SEOUL, South Korea — North Korea’s leader, ..."


# Extracting the key words from the content description.

In [105]:
# initializing the new column
df2['Key_words'] = ""

for index, row in df2.iterrows():
    content = row['content']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(content)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df2.drop(columns = ['content'], inplace = True)

In [106]:
#set title as the index 
df2.set_index('title', inplace = True)
df2.head()

,publication,author,Key_words
title,,,
House Republicans Fret About Winning Their Health Care Suit - The New York Times,newyorktimes,carlhulse,"[important, congress, never, appropriated, app..."
Rift Between Officers and Residents as Killings Persist in South Bronx - The New York Times,newyorktimes,benjaminmuellerandalbaker,"[someone, might, people, dealing, drugs, outsi..."
"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial Bias, Dies at 106 - The New York Times",newyorktimes,margalitfox,"[make, art, bring, wake, claimed, obliged, dis..."
"Among Deaths in 2016, a Heavy Toll in Pop Music - The New York Times",newyorktimes,williammcdonald,"[played, bed, gratitude, acting, leading, lady..."
Kim Jong-un Says North Korea Is Preparing to Test Long-Range Missile - The New York Times,newyorktimes,choesang-hun,"[conduct, allows, kim, ’, keep, soon, 20, ball..."


In [107]:
#creating bag of words
df2['bag_of_words'] = ''
columns = df2.columns
for index, row in df2.iterrows():
    words = ''
    for col in columns:
        if col != 'publication':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df2.drop(columns = [col for col in df2.columns if col!= 'bag_of_words'], inplace = True)

In [108]:
df2.head()

,bag_of_words
title,
House Republicans Fret About Winning Their Health Care Suit - The New York Times,newyorktimes c a r l h u l s e important congr...
Rift Between Officers and Residents as Killings Persist in South Bronx - The New York Times,newyorktimes b e n j a m i n m u e l l e r a n...
"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial Bias, Dies at 106 - The New York Times",newyorktimes m a r g a l i t f o x make art br...
"Among Deaths in 2016, a Heavy Toll in Pop Music - The New York Times",newyorktimes w i l l i a m m c d o n a l d pla...
Kim Jong-un Says North Korea Is Preparing to Test Long-Range Missile - The New York Times,newyorktimes c h o e s a n g - h u n conduct a...


In [109]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df2['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# listto be used later to match the indexes
indices = pd.Series(df2.index)
indices[:5]

0    House Republicans Fret About Winning Their Hea...
1    Rift Between Officers and Residents as Killing...
2    Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...
3    Among Deaths in 2016, a Heavy Toll in Pop Musi...
4    Kim Jong-un Says North Korea Is Preparing to T...
Name: title, dtype: object

# generating the cosine similarity matrix

In [110]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

MemoryError: 

In [61]:
df2['publication'] = df2['publication'].map(lambda x: x.split(' '))
df2['author'] = df['author'].map(lambda x: x.split(' '))

AttributeError: 'list' object has no attribute 'split'

In [58]:
for index, row in df2.iterrows():
    row['publication'] = [x.lower().replace(' ','') for x in row['publication']]
    #row['author'] = ''.join(row['author']).lower()

In [83]:
df2.dropna(how='any',axis=0)     

,Unnamed: 0,id,title,publication,author,date,year,month,url,content


In [82]:
print(df2.author.value_counts())

nan                                                              6306
Breitbart News                                                   1559
Pam Key                                                          1282
Charlie Spiering                                                  928
Jerome Hudson                                                     806
John Hayward                                                      747
Daniel Nussbaum                                                   735
AWR Hawkins                                                       720
Ian Hanchett                                                      647
Joel B. Pollak                                                    624
Alex Swoyer                                                       604
Warner Todd Huston                                                545
Jeff Poor                                                         505
Trent Baker                                                       457
Breitbart London    

In [59]:
df2.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [50]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

# putting the genres in a list of words
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

df['Director'] = df['Director'].map(lambda x: x.split(' '))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

SyntaxError: unexpected EOF while parsing (<ipython-input-50-92d206d238f1>, line 5)

In [34]:
df2.dtypes

title          object
publication    object
author         object
content        object
dtype: object